In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation


In [2]:
import pandas as pd
from surprise import Dataset
from surprise import Reader

from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split as train_test_split_surprise
from surprise import Dataset, SVD
from surprise.model_selection import cross_validate, GridSearchCV
from sklearn.model_selection import train_test_split

import numpy as np

In [3]:
raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
raw_interactions.shape

(1132367, 5)

### Create Train/Test/validation from raw

In [4]:
TRAIN__Validation_SIZE = 0.8
TEST_SIZE = 0.2

In [5]:
train_validation, test_set = train_test_split(raw_interactions, test_size=TEST_SIZE)
train_validation.shape, test_set.shape

((905893, 5), (226474, 5))

In [6]:
train_validation.to_csv("../../data/recommendation/ad_interaction_train.csv")
test_set.to_csv("../../data/recommendation/ad_interaction_test.csv")

In [7]:
abhinav_test = pd.read_csv("../../data/recommendation/df_collaborative_test.csv")
abhinav_test.shape

(80212, 4)

In [8]:
abhinav_test.columns

Index(['Unnamed: 0', 'rating', 'user_id', 'recipe_id'], dtype='object')

In [9]:
abhinav_test.dtypes

Unnamed: 0    int64
rating        int64
user_id       int64
recipe_id     int64
dtype: object

In [23]:
test_set.dtypes

user_id       int64
recipe_id     int64
date         object
rating        int64
review       object
dtype: object

In [10]:
len(set(abhinav_test.user_id)-set(train_validation.user_id))

10512

In [11]:
set(abhinav_test.user_id).intersection(set(train_validation.user_id))

{1535,
 1581,
 1634,
 1676,
 1755,
 1773,
 1792,
 1891,
 1938,
 1962,
 1986,
 2008,
 2054,
 2059,
 2070,
 2073,
 2095,
 2148,
 2178,
 2205,
 2246,
 2310,
 2312,
 2378,
 2447,
 2525,
 2536,
 2555,
 2557,
 2585,
 2586,
 2595,
 2611,
 2625,
 2693,
 2695,
 2703,
 2781,
 2811,
 2848,
 2957,
 2962,
 2971,
 2985,
 2998,
 2999,
 3090,
 3101,
 3111,
 3114,
 3153,
 3187,
 3198,
 3201,
 3205,
 3228,
 3248,
 3253,
 3288,
 3292,
 3307,
 3337,
 3385,
 3420,
 3460,
 3484,
 3504,
 3505,
 3514,
 3518,
 3525,
 3539,
 3566,
 3572,
 3575,
 3577,
 3607,
 3619,
 3622,
 3627,
 3634,
 3652,
 3657,
 3672,
 3674,
 3678,
 3682,
 3731,
 3735,
 3775,
 3793,
 3794,
 3807,
 3810,
 3813,
 3820,
 3824,
 3844,
 3858,
 3861,
 3895,
 3934,
 3962,
 3980,
 3981,
 3989,
 4005,
 4039,
 4043,
 4046,
 4073,
 4074,
 4093,
 4096,
 4098,
 4099,
 4105,
 4120,
 4141,
 4142,
 4149,
 4150,
 4151,
 4155,
 4166,
 4174,
 4177,
 4203,
 4241,
 4242,
 4262,
 4272,
 4287,
 4291,
 4309,
 4312,
 4317,
 4322,
 4335,
 4337,
 4340,
 4347,
 4353,

In [12]:
len(set(abhinav_test.recipe_id)-set(train_validation.recipe_id))

9906

In [13]:
abhinav_test[abhinav_test.user_id == 1535]

,Unnamed: 0,rating,user_id,recipe_id
1675,207547,5,1535,2356
43470,284632,5,1535,7880
61671,121423,4,1535,4836


In [14]:
sum(train_validation[train_validation.user_id == 1535].recipe_id == 2356),\
sum(train_validation[train_validation.user_id == 1535].recipe_id == 7880),\
sum(train_validation[train_validation.user_id == 1535].recipe_id == 4836)

(0, 0, 0)

In [15]:
pure_test = pd.merge(abhinav_test, train_validation, on=['user_id','recipe_id'], how="outer", indicator=True
              ).query('_merge=="left_only"')
pure_test.shape

(80212, 8)

In [28]:
pure_test

,Unnamed: 0,rating_x,user_id,recipe_id,date,rating_y,review,_merge
0,333675.0,5.0,6637,10576,NaN,NaN,NaN,left_only
1,375418.0,5.0,177,4710,NaN,NaN,NaN,left_only
2,143241.0,0.0,7440,5722,NaN,NaN,NaN,left_only
3,58583.0,5.0,3710,13365,NaN,NaN,NaN,left_only
4,6765.0,5.0,9440,17528,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...
80207,241085.0,4.0,2941,19168,NaN,NaN,NaN,left_only
80208,87177.0,5.0,11811,20222,NaN,NaN,NaN,left_only
80209,251194.0,5.0,275,10397,NaN,NaN,NaN,left_only
80210,107754.0,3.0,10745,2147,NaN,NaN,NaN,left_only


### Train SVD matrix Factorisation

In [ ]:
reader = Reader(rating_scale=(1, 5))
cv_data = Dataset.load_from_df(train_validation[["user_id", "recipe_id", "rating"\
                                                                           ]], reader)

In [ ]:
param_grid = {"n_factors":[20, 50] ,"n_epochs": [10, 15], "lr_all": [0.002, 0.005]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5, n_jobs = -2)

gs.fit(cv_data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

print(gs.best_params["mae"])

In [ ]:
cv_results_df = pd.DataFrame.from_dict(gs.cv_results)
cv_results_df

### Test RMSE

In [ ]:
#Build model on full cv set
algo = gs.best_estimator["rmse"]
algo.fit(cv_data.build_full_trainset())

In [ ]:
test_set_surprise = Dataset.load_from_df(test_set[["user_id", "recipe_id", "rating"]], reader)
_, test_set_surprise = train_test_split_surprise(test_set_surprise, test_size=1.0)

In [ ]:
len(test_set_surprise)

In [ ]:
test_set_surprise[:5]

In [ ]:
predictions = algo.test(test_set_surprise)
accuracy.rmse(predictions, verbose=True)

### Analyse the latent features

In [ ]:
algo.trainset.n_users, algo.trainset.n_items

In [ ]:
user_matrix = algo.pu
user_matrix.shape

In [ ]:
movie_matrix = algo.qi
movie_matrix.shape

In [ ]:
movie_matrix

In [ ]:
cv_data

In [ ]:
movie_mat_df = pd.DataFrame(movie_matrix)
movie_mat_df.columns = [f"latent_{i+1}" for i in range(len(movie_mat_df.columns))]
movie_mat_df["recipe_id"] = cv_data.df.recipe_id

In [ ]:
recipe_metadata = pd.read_csv("../../data/recommendation/RAW_recipes.csv")
recipe_metadata.shape

In [ ]:
recipe_metadata.head()

In [ ]:
movie_mat_join_recipe_meta = recipe_metadata.merge(movie_mat_df,left_on = "id",\
                                                 right_on = "recipe_id")
movie_mat_join_recipe_meta.shape

In [ ]:
latent_cols = [x for x in movie_mat_join_recipe_meta.columns if "latent" in x]
latent_cols

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(movie_mat_join_recipe_meta[latent_cols].corr())
plt.show()

In [ ]:
def _my_corr(x):
    df_corr = movie_mat_join_recipe_meta[[x,"minutes","n_ingredients","n_steps"]].corr()
    df_corr = df_corr.drop(columns = [x]).iloc[:1]
    return (df_corr)

In [ ]:
_my_corr(latent_i)

In [ ]:
corr_result = pd.DataFrame()
for latent_i in latent_cols:
    corr_result = pd.concat([corr_result,_my_corr(latent_i)],axis=0)

In [ ]:
plt.matshow(corr_result.T)
cb = plt.colorbar()
plt.show()

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, n_iter=500, verbose=3, random_state=1)
recipe_embedding = tsne.fit_transform(movie_matrix)
projection = pd.DataFrame(columns=['x', 'y'], data=recipe_embedding)
# projection['title'] = books_metadata['original_title']

In [ ]:
projection.head()

In [ ]:
plt.scatter(projection["x"],projection["y"],alpha = 0.25)

train_validation, test_set = train_test_split_surprise(full_data, test_size=0.15)

train_validation.n_ratings, len(test_set), train_validation.n_ratings + len(test_set)

train_validation.n_items, train_validation.n_users

### Build SVD Matrix Factorisation

svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, train_validation, measures=['RMSE', 'MAE'], cv=10, verbose=True)

dir(train_validation)